In [1]:
import pandas as pd
import glob
from sklearn.externals import joblib as jl
import os
from urllib.parse import urlparse
import numpy as np
import tldextract

%matplotlib inline

In [2]:
def ref_domains_extract(data_path):
    domains = os.listdir(data_path)
    data = pd.DataFrame(columns=['domain', 'ref_domain', 'source'])
    doms =[]
    print('Done:')
    for dom in domains:
        files = glob.glob("{}/{}/*.csv".format(data_path, dom))
        for f in files:
            if 'backlinks-subdomains-recent' in f:
                links = pd.read_csv(f, usecols=['Referring Page URL'], sep='\t', encoding='utf-16')['Referring Page URL']
                source = 'AHREFS'

            if 'download_fresh_' in f:
                links = pd.read_csv(f, usecols=['Source URL'])['Source URL']
                source = 'MAJESTIC'

            if 'ExternalLinks_SampleLinks' in f:
                links = pd.read_csv(f, usecols=['Links'])['Links']
                source = 'GOOGLE'
            
            links = links.apply(lambda x: '{uri.netloc}'.format(uri=urlparse(x)))
#             links = links.apply(lambda x: tldextract.extract(x).registered_domain)
            df = pd.DataFrame({'ref_domain': links.unique()})
            df['domain'] = dom
            df['source'] = source
            data = data.append(df, ignore_index=True)
    
        doms.append(dom)
    print(len(doms))
    jl.dump(data, 'data/ref_domains_crawl2.jl')

In [3]:
# %%time
# data_path = 'data/GSC Ahrefs Majestic'
# ref_domains_extract(data_path)

Done:
104
Wall time: 9min 58s


In [ ]:
# data0 = jl.load('data/ref_domains_crawl2.jl').drop_duplicates().reset_index(drop=True)
# data0 = data0[data0.ref_domain != '']
# data0.head()

In [4]:
# data0.to_csv('data/data0.csv')

In [2]:
pd.read_csv('data/out.csv', header=1, index_col=0).head()

,domain,ref_domain,source,ref_domain.1,Unnamed: 5,ref_domain.2
0,10edges.com,3dprintingninja.blogspot.com.ee,GOOGLE,3dprintingninja.blogspot.com.ee,3dprintingninja.blogspot,com.ee
1,10edges.com,3dprintingninja.blogspot.com,GOOGLE,3dprintingninja.blogspot.com,3dprintingninja.blogspot,com
2,10edges.com,3dprintingninja.blogspot.fr,GOOGLE,3dprintingninja.blogspot.fr,3dprintingninja.blogspot,fr
3,10edges.com,3dprintingninja.blogspot.in,GOOGLE,3dprintingninja.blogspot.in,3dprintingninja.blogspot,in
4,10edges.com,3dprintingninja.blogspot.nl,GOOGLE,3dprintingninja.blogspot.nl,3dprintingninja.blogspot,nl


In [3]:
data0 = pd.read_csv('data/out.csv', header=1, index_col=0)[['domain', 'ref_domain.1', 'source']]
data0.rename(columns={'ref_domain.1': 'ref_domain'}, inplace=True)
data0.head()

,domain,ref_domain,source
0,10edges.com,3dprintingninja.blogspot.com.ee,GOOGLE
1,10edges.com,3dprintingninja.blogspot.com,GOOGLE
2,10edges.com,3dprintingninja.blogspot.fr,GOOGLE
3,10edges.com,3dprintingninja.blogspot.in,GOOGLE
4,10edges.com,3dprintingninja.blogspot.nl,GOOGLE


In [4]:
data0.loc[[0,1,2,3,4,13, 36, 37, 38, 39]]

,domain,ref_domain,source
0,10edges.com,3dprintingninja.blogspot.com.ee,GOOGLE
1,10edges.com,3dprintingninja.blogspot.com,GOOGLE
2,10edges.com,3dprintingninja.blogspot.fr,GOOGLE
3,10edges.com,3dprintingninja.blogspot.in,GOOGLE
4,10edges.com,3dprintingninja.blogspot.nl,GOOGLE
13,10edges.com,carthage.edu,GOOGLE
36,10edges.com,dcc.edu,GOOGLE
37,10edges.com,droneflyers.com,GOOGLE
38,10edges.com,ec.edu,GOOGLE
39,10edges.com,financeservicesfinder.com,GOOGLE


### consider all unique ref.domains found in 3 sources as 100%, then what % from all these ref.domains each individual source has found?

Method 1: calculate % of unique ref.domains for each of 104 sites separately, and then take the average and median values across all 104 sites

In [5]:
binarize = pd.get_dummies(data0['source'])
data = pd.concat([data0, binarize], axis=1).drop(['source'], axis=1)
data.head()

,domain,ref_domain,AHREFS,GOOGLE,MAJESTIC
0,10edges.com,3dprintingninja.blogspot.com.ee,0,1,0
1,10edges.com,3dprintingninja.blogspot.com,0,1,0
2,10edges.com,3dprintingninja.blogspot.fr,0,1,0
3,10edges.com,3dprintingninja.blogspot.in,0,1,0
4,10edges.com,3dprintingninja.blogspot.nl,0,1,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169424 entries, 0 to 169423
Data columns (total 5 columns):
domain        169424 non-null object
ref_domain    169424 non-null object
AHREFS        169424 non-null uint8
GOOGLE        169424 non-null uint8
MAJESTIC      169424 non-null uint8
dtypes: object(2), uint8(3)
memory usage: 9.4+ MB


In [7]:
data = data.groupby(['domain', 'ref_domain'], as_index=False).agg({'AHREFS': max,
                                                                   'GOOGLE': max,
                                                                   'MAJESTIC': max})
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94600 entries, 0 to 94599
Data columns (total 5 columns):
domain        94600 non-null object
ref_domain    94600 non-null object
MAJESTIC      94600 non-null uint8
AHREFS        94600 non-null uint8
GOOGLE        94600 non-null uint8
dtypes: object(2), uint8(3)
memory usage: 2.4+ MB


In [8]:
data.head()

,domain,ref_domain,MAJESTIC,AHREFS,GOOGLE
0,10edges.com,3dprintingninja.blogspot.com,0,0,1
1,10edges.com,3dprintingninja.blogspot.com.ee,0,0,1
2,10edges.com,3dprintingninja.blogspot.fr,0,0,1
3,10edges.com,3dprintingninja.blogspot.in,0,0,1
4,10edges.com,3dprintingninja.blogspot.nl,0,0,1


In [9]:
domain_ave = data.groupby(['domain']).agg({'AHREFS': np.mean,
                                            'GOOGLE': np.mean,
                                            'MAJESTIC': np.mean})
domain_ave.head()

,MAJESTIC,AHREFS,GOOGLE
domain,,,
10edges.com,0.390909,0.490909,0.709091
4quotes4me.co.uk,0.511468,0.582569,0.324541
Exploringthisrock.com,0.383333,0.405556,0.538889
aaronneo.com.au,0.121622,0.378378,0.810811
aionhill.com,0.317895,0.601053,0.587368


In [10]:
domain_ave.mean().to_frame('mean').join(domain_ave.median().to_frame('median')).sort_index()

,mean,median
AHREFS,0.516592,0.521605
GOOGLE,0.606202,0.632843
MAJESTIC,0.374908,0.382015


Method 2: consider all 104 domains as one website, and calculate the totals from all data that we have

In [11]:
data_one = data.groupby(['ref_domain']).agg({'AHREFS': max,
                                             'GOOGLE': max,
                                             'MAJESTIC': max})
data_one.head()

,MAJESTIC,AHREFS,GOOGLE
ref_domain,,,
0-21.co.uk,0,1,0
0-3-0.com,0,1,0
000.nl,1,1,1
000a.biz,0,0,1
000webhostapp.com,0,1,0


In [12]:
data_one.mean().sort_index().to_frame('mean')

,mean
AHREFS,0.582489
GOOGLE,0.581244
MAJESTIC,0.475362


### Intersections for Venn

In [13]:
venn = data0.drop(['domain'], axis=1).drop_duplicates().reset_index(drop=True)

In [14]:
len(venn)

118518

In [15]:
venn.source.value_counts().sort_index()

AHREFS      42118
GOOGLE      42028
MAJESTIC    34372
Name: source, dtype: int64

In [16]:
venn.to_csv('data/for_venn.csv')

In [17]:
# unique to AHREFS
print ('unique to AHREFS: {}'.format(len(data[(data.AHREFS == 1) &\
                                (data.MAJESTIC == 0) &\
                                (data.GOOGLE == 0)])))

# unique to MAJESTIC
print ('unique to MAJESTIC: {}'.format(len(data[(data.AHREFS == 0) &\
                                          (data.MAJESTIC == 1) &\
                                          (data.GOOGLE == 0)])))

# unique to GOOGLE
print ('unique to GOOGLE: {}'.format(len(data[(data.AHREFS == 0) &\
                                          (data.MAJESTIC == 0) &\
                                          (data.GOOGLE == 1)])))

# AHREFS and MAJESTIC
print ('AHREFS & MAJESTIC: {}'.format(len(data[(data.AHREFS == 1) &\
                                                       (data.MAJESTIC == 1) &\
                                                       (data.GOOGLE == 0)])))

# AHREFS and GOOGLE
print ('AHREFS & GOOGLE: {}'.format(len(data[(data.AHREFS == 1) &\
                                                     (data.MAJESTIC == 0) &\
                                                     (data.GOOGLE == 1)])))

# MAJESTIC and GOOGLE
print ('MAJESTIC & GOOGLE: {}'.format(len(data[(data.AHREFS == 0) &\
                                                       (data.MAJESTIC == 1) &\
                                                       (data.GOOGLE == 1)])))

# AHREFS and MAJESTIC and GOOGLE
print ('AHREFS & MAJESTIC & GOOGLE: {}'.format(len(data[(data.AHREFS == 1) &\
                                                              (data.MAJESTIC == 1) &\
                                                              (data.GOOGLE == 1)])))

unique to AHREFS: 19043
unique to MAJESTIC: 14271
unique to GOOGLE: 23730
AHREFS & MAJESTIC: 10036
AHREFS & GOOGLE: 9968
MAJESTIC & GOOGLE: 3951
AHREFS & MAJESTIC & GOOGLE: 13601


In [18]:
data_one.sum().sort_index()

AHREFS      42118
GOOGLE      42028
MAJESTIC    34372
dtype: int64

In [19]:
ingoogle = data[(data.AHREFS == 0) & (data.GOOGLE == 1)].drop(['MAJESTIC', 'GOOGLE', 'AHREFS'], axis=1)
ingoogle.head()

,domain,ref_domain
0,10edges.com,3dprintingninja.blogspot.com
1,10edges.com,3dprintingninja.blogspot.com.ee
2,10edges.com,3dprintingninja.blogspot.fr
3,10edges.com,3dprintingninja.blogspot.in
4,10edges.com,3dprintingninja.blogspot.nl


In [20]:
inmajestic = data[(data.AHREFS == 0) & (data.MAJESTIC == 1)].drop(['MAJESTIC', 'GOOGLE', 'AHREFS'], axis=1)
inmajestic.head()

,domain,ref_domain
9,10edges.com,americanbookreview.org
16,10edges.com,bju.edu
20,10edges.com,carthage.edu
23,10edges.com,coolpot.com
29,10edges.com,dronestoriesetc.com


In [21]:
ingoogle.to_csv('data/ingoogle.csv',index=False)
inmajestic.to_csv('data/inmajestic.csv',index=False)